In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import scipy.stats as st
from pprint import pprint
import time
from scipy.stats import linregress
import reverse_geocoder as rg

In [2]:

# Set base url parameters
base_url = "https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson"
magnitude = "minmagnitude=5"
limit = "limit=19999"
year=10

# set up lists to hold reponse info
time_ls = []
lat_ls = []
lng_ls = []
depth_ls = []
mag_ls = []
mmi_ls = []
sig_ls = []
felt_ls = []
place_ls = []

# iterate through years
for x in range (0,11):
    time = f"starttime=20{year}-01-01&endtime=20{year}-12-31"
    query_url = (f"{base_url}&{time}&{limit}")
    response = requests.get(query_url).json()

    # Append data to appropriate list
    for i in range (0, len(response["features"])):
        mag_ls.append(response["features"][i]["properties"]["mag"])
        place_ls.append(response["features"][i]["properties"]["place"])
        time_ls.append(response["features"][i]["properties"]["time"])
        felt_ls.append(response["features"][i]["properties"]["felt"])
        mmi_ls.append(response["features"][i]["properties"]["mmi"])
        sig_ls.append(response["features"][i]["properties"]["sig"])
        lat_ls.append(response["features"][i]["geometry"]["coordinates"][1])
        lng_ls.append(response["features"][i]["geometry"]["coordinates"][0])
        depth_ls.append(response["features"][i]["geometry"]["coordinates"][2])
    # counter for next year
    
    year=year+1
    

In [3]:
# create dictionary
data = {
    "Time": time_ls,
    "Lat": lat_ls,
    "Lng": lng_ls,
    "Depth": depth_ls,
    "Mag": mag_ls,
    "MMI": mmi_ls,
    "Sig": sig_ls,
    "Felt": felt_ls,
    "Place": place_ls
}
# create dataframe
data_df = pd.DataFrame(data)

#convert timestamp to dates
data_df["Time"] = pd.to_datetime(data_df["Time"], unit="ms")
data_df["Month"] = pd.DatetimeIndex(data_df["Time"]).month
data_df["Day"] = pd.DatetimeIndex(data_df["Time"]).day
data_df["Year"] = pd.DatetimeIndex(data_df["Time"]).year

# #add country
# data_df["Country"] = ""

# for index, row in data_df.iterrows():
#     # get lat, lng from df
#     lat = row["Lat"]
#     lng = row["Lng"]
#     coordinates = (lat,lng)
#     result = rg.search(coordinates)
#     data_df.loc[index,"Country"] = result[0]["cc"]

In [7]:
# Print dataframe

data_df= data_df[["Lat","Lng","Depth","Mag","MMI","Sig","Felt","Place","Time","Month","Day","Year"]]

data_df

,Lat,Lng,Depth,Mag,MMI,Sig,Felt,Place,Time,Month,Day,Year
0,38.819832,-122.822502,0.904,0.56,NaN,5,NaN,Northern California,2010-12-30 23:55:30.100,12,30,2010
1,35.302000,-92.334333,3.500,2.10,NaN,68,NaN,"8km NNE of Greenbrier, Arkansas",2010-12-30 23:54:43.770,12,30,2010
2,35.315667,-92.325167,2.050,2.30,NaN,81,NaN,"10km NNE of Greenbrier, Arkansas",2010-12-30 23:51:53.500,12,30,2010
3,32.998500,-116.350333,9.098,1.37,NaN,29,NaN,"25km ESE of Julian, CA",2010-12-30 23:49:00.610,12,30,2010
4,-31.830000,-178.135000,35.000,5.00,NaN,385,NaN,Kermadec Islands region,2010-12-30 23:47:03.930,12,30,2010
...,...,...,...,...,...,...,...,...,...,...,...,...
219984,63.200200,-151.317600,2.300,0.90,NaN,12,NaN,"43 km SSE of Denali National Park, Alaska",2020-11-06 05:30:20.995,11,6,2020
219985,51.300900,-176.511300,25.600,2.10,NaN,68,NaN,"64 km S of Adak, Alaska",2020-11-06 05:28:13.270,11,6,2020
219986,62.960100,-150.821400,98.100,1.40,NaN,30,NaN,"51 km N of Petersville, Alaska",2020-11-06 05:18:27.127,11,6,2020
219987,36.527333,-98.990667,7.710,1.14,NaN,20,NaN,"9 km NNE of Quinlan, Oklahoma",2020-11-06 05:15:59.969,11,6,2020


In [6]:
data_df.to_csv("output_data/earthquake_data_2010-20.csv", index= False)